### 01. 통계분석

##### Admission 데이터 변수의 설명은 아래와 같다.

<table>
    <thead>
        <tr>
            <th>변수</th>
            <th>데이터 형태</th>
            <th>설명</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>GRE</td>
            <td>수치형</td>
            <td>GRE점수</td>
        </tr>
        <tr>
            <td>TOEFL</td>
            <td>수치형</td>
            <td>TOEFL점수</td>
        </tr>
        <tr>
            <td>Univ_Rating</td>
            <td>수치형</td>
            <td>대학교 등급점수(1~5 사이의 숫자)</td>
        </tr>
        <tr>
            <td>SOP</td>
            <td>수치형</td>
            <td>자기소개서 점수</td>
        </tr>
        <tr>
            <td>LOR</td>
            <td>수치형</td>
            <td>추천서 점수</td>
        </tr>
        <tr>
            <td>CGPA</td>
            <td>수치형</td>
            <td>평점평균점수</td>
        </tr>
        <tr>
            <td>Research</td>
            <td>범주형</td>
            <td>연구 실적 유무(0: 없음, 1: 있음)</td>
        </tr>
        <tr>
            <td>Chance_of_Admit</td>
            <td>수치형</td>
            <td>입학 허가 확률</td>
        </tr>
    </tbody>
</table>

#### 1) 종속변수인 Chance_of_Admit(입학 허가 확률)와 독립변수(GRE, TOEFL, Univ_Rating, SOP, LOR, CGPA)에 대해 피어슨 상관계수를 이용한 상관관계 분석을 수행하고 그래프를 이용하여 분석결과를 설명하시오.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data/모의고사 2회/Admission.csv", encoding="utf-8")
df

In [ ]:
#컬럼 값에 있는 공백 제거
new_cols = []
for col in df.columns:
    new_cols.append(col.strip())

df.columns = new_cols

In [ ]:
#결측치 있는지 체크
for col in df.columns:
    isna = False
    isna_df = df[[col]].isna().query(col+" == True")
    if len(isna_df) > 0:
        isna = True
    print("{} : {} / len : {}".format(col, isna, len(isna_df)))

In [ ]:
#상관계수 산출
var_list = []
for col in df.columns:
    if col != "Research":
        var_list.append(col)

df[var_list].corr()

In [ ]:
import scipy.stats as stats

#scipy로 상관계수 계산 & 검정
for col in var_list:
    r = stats.pearsonr(df["Chance_of_Admit"], df[col])
    sns.regplot(x=df[col], y=df["Chance_of_Admit"])
    plt.show()
    print("{} - Chance_of_Admit".format(col))
    print(" └상관계수 : {}, p-value : {}".format(r[0], r[1]), end="\n\n")

print("모두 양의 상관관계, 검정결과도 적합")

In [ ]:
sns.pairplot(df[var_list], kind="reg", height=5)

In [ ]:
plt.figure(figsize=(12, 10))
plt.rcParams["font.size"] = 14
colormap = plt.cm.PuBu
sns.heatmap(df[var_list].corr(), cmap=colormap, annot = True, annot_kws = {"size" : 16})
plt.show()

#### 2) GRE, TOEFL, Univ_Rating, SOP, LOR, CGPA, Research가 Chance_of_Admit에 영향을 미치는지 알아보는 회귀분석을 단계적 선택법을 사용하여 수행하고 결과를 해석하시오.

참고 : https://no17.tistory.com/195 , https://mindscale.kr/course/basic-stat-python/13/

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn

model = None

In [ ]:
def reg(df, dep_var, ind_vars):
    global model
    ind_vars_str = " + ".join(ind_vars)
    model = smf.ols(formula=dep_var+" ~ "+ind_vars_str, data=df)
    print(model.fit().summary())

In [ ]:
dep_var = "Chance_of_Admit"
ind_vars = ["GRE", "TOEFL", "Univ_Rating", "SOP", "LOR", "CGPA", "Research"]

reg(df, dep_var, ind_vars)

In [ ]:
#다중공선성을 계산한다.
# print(model.exog_names) #독립변수 리스트 출력
factor1_vif = variance_inflation_factor(model.exog, 1) #첫번째 독립변수의 VIF를 계산해 준다.

vif_dic = []
for i, var in enumerate(model.exog_names):
    if i == 0:
        continue
    this_dic = {
        "독립변수" : model.exog_names[i],
        "VIF" : variance_inflation_factor(model.exog, i)
    }
    vif_dic.append(this_dic)
df_vif = pd.DataFrame(vif_dic)
display(df_vif)
print("CGPA의 VIF가 5 이상으로, 다중공선성 주의 필요")
#VIF가 10이 넘으면 다중공선성 있다고 판단하며 5가 넘으면 주의할 필요가 있는 것으로 봅니다.
#독립 변수 a와 b가 서로 상관 관계가 있다고 했을 때 두 변수 모두 VIF가 높습니다.
#어느 하나만 VIF가 높은 경우는 없습니다.
#박수도 오른손과 왼손이 있어야 칠 수 있듯이 서로 연관 있는 변수끼리 VIF가 높습니다.

In [ ]:
#전진 선택법 함수를 만들어 보자 (단계적 선택법도 옵션에 포함)
#참고 : https://heejeongchoi.github.io/hydejack/2018-10-23-Supervised-Dimension-Reduction/
def forward_selection(df, dep_var, ind_vars, select_criteria = "AIC", stepwise=False):
    if stepwise:
        #후진제거법으로 제거되는 변수를 먼저 찾아 놓는다.
        eliminated_vars = backward_elimination(df, dep_var, ind_vars, "AIC", "e")
    candidate_vars = ind_vars.copy()
    selected_vars = [] #전진선택법으로 선택된 변수들
    alpha = 0.05 #새롭게 편입된 변수의 회귀계수 p-value가 0.05보다 작을 경우에만 선택
    
    while len(candidate_vars) > 0: #후보 변수 리스트가 모두 소거될 때까지 반복한다.
        candidate_vars_reg_result = []
        for candidate in candidate_vars:
            #새로운 변수가 추가되었을 때 회귀모형의 AIC와 r-square값을 구해본다.
            this_ind_vars_str = candidate
            if len(selected_vars) > 0:
                this_ind_vars_str = " + ".join(selected_vars) + " + " + candidate
            this_model_fit = smf.ols(dep_var+" ~ "+this_ind_vars_str, data=df).fit()
            this_result = {
                "var" : candidate,
                "AIC" : this_model_fit.aic,
                "rsquared_adj" : this_model_fit.rsquared_adj,
                "p-value" : this_model_fit.pvalues[candidate]
            }
            candidate_vars_reg_result.append(this_result)
        result = pd.DataFrame(candidate_vars_reg_result)
        ascending = True
        if select_criteria == "rsquared_adj":
            ascending = False
        result.sort_values(by=select_criteria, ascending=ascending, inplace=True)
        selected_var = result.iloc[0,[0]].values[0]
        selected_var_pvalue = result.iloc[0,[3]].values[0]
        if selected_var_pvalue > alpha:
            break
        selected_vars.append(selected_var)
        candidate_vars.remove(selected_var)
        if stepwise:
            #selected_vars 중에 eliminated_var에 해당하는 것이 있다면 제거한다
            if len(selected_vars) > 2 and len(eliminated_vars) > 0:
                for ev in eliminated_vars:
                    if ev in selected_vars:
                        selected_vars.remove(ev)
    print("후보 독립변수들 : {}".format(ind_vars))
    if stepwise:
        print("단계적 선택법 선택변수 : {}".format(selected_vars))
    else:
        print("전진선택법 선택변수 : {}".format(selected_vars))
    return selected_vars

forward_selected_ind_vars = forward_selection(df, dep_var, ind_vars, "AIC")

forward_selected_ind_vars_str = " + ".join(forward_selected_ind_vars)
model = smf.ols(dep_var+" ~ "+forward_selected_ind_vars_str, data=df)
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
#후진 제거법 함수를 만들어 보자
#참고 : https://heejeongchoi.github.io/hydejack/2018-10-23-Supervised-Dimension-Reduction/
def backward_elimination(df, dep_var, ind_vars, select_criteria = "AIC", return_case = "selected_vars"):
    alpha = 0.05
    selected_vars = ind_vars.copy()
    eliminated_vars = []
    done = False
    
    while done is False:
        #step1 : 변수 하나씩을 제외한 n개의 모델을 만들어낸다. (n : 변수 갯수)
        model_result_list = []
        for var in ind_vars:
            var_list = ind_vars.copy()
            if len(eliminated_vars)>0:
                var_list.remove(eliminated_vars)
            var_list.remove(var)
            this_ind_vars_str = " + ".join(var_list)
            this_model_fit = smf.ols(dep_var+" ~ "+this_ind_vars_str, data=df).fit()
            this_result = {
                "selected var" : this_ind_vars_str,
                "eliminated var" : var,            
                "AIC" : this_model_fit.aic,
                "rsquared_adj" : this_model_fit.rsquared_adj
            }
            model_result_list.append(this_result)
        result = pd.DataFrame(model_result_list)
        #step2 : n개 모델들 중 가장 좋은 결과를 보인 모델을 선택하고, 제외 후보 변수를 찾는다.
        ascending = True
        if select_criteria == "rsquared_adj":
            ascending = False
        result.sort_values(by=select_criteria, ascending=ascending, inplace=True)
        candidate_var = result.iloc[0, [1]].values[0]
        #step3 : 제외 후보 변수로 fitting하여 p-value를 알아보고 유의하다면 종료한다.
        candidate_model_fit = smf.ols(dep_var+" ~ "+candidate_var, data=df).fit()
        if candidate_model_fit.pvalues[candidate_var] > alpha: #유의하지 않다면
            eliminated_vars.append(candidate_var) #제거확정
            selected_vars.remove(candidate_var)
        else: #유의하다면 종료한다
            done = True
    if return_case == "s":
        print("후보 독립변수들 : {}".format(ind_vars))
        print("후진제거법 선택변수 : {}".format(selected_vars))
        return selected_vars
    elif return_case == "e":
        return eliminated_vars
    
backward_selected_ind_vars = backward_elimination(df, dep_var, ind_vars, "AIC", "s")

backward_selected_ind_vars_str = " + ".join(backward_selected_ind_vars)
model = smf.ols(dep_var+" ~ "+backward_selected_ind_vars_str, data=df)
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
stepwise_selected_ind_vars = forward_selection(df, dep_var, ind_vars, "AIC", True)

stepwise_selected_ind_vars_str = " + ".join(stepwise_selected_ind_vars)
model = smf.ols(dep_var+" ~ "+stepwise_selected_ind_vars_str, data=df)
model_fit = model.fit()
print(model_fit.summary())

#### 3) 단계 선택법을 사용해 변수를 선택한 후 새롭게 생성한 회귀모형에 대한 잔차분석을 수행하고, 그래프를 활용하여 결과를 해석하시오.